In [13]:
import re 
import os
import tensorflow as tf
import ROOT

from tensorflow.keras.callbacks import CSVLogger

import sys
sys.path.insert(0, "../python")
import InputsProducer
from SignalSelectionModel import HHModel

In [15]:
def ListToVector(l, elem_type):
    v = ROOT.std.vector('string')()
    for x in l:
        v.push_back(x)
    return v

In [16]:
files = []
for r, d, f in os.walk('/data/dido/NN_samples'):
    for file in f:
        if re.match(r'.*2018.*ggHH_NonRes\.root', file):
            files.append(os.path.join(r, file))
# print(files)

In [17]:
file_name = '/data/dido/new_samples/NN_samples/VBFToBulkGravitonToHHTo2B2Tau_M-400_narrow_eTau_2018_VBFHH_Res.root'
# file_name = '/data/dido/new_samples/NN_samples/*.root'

In [18]:
%%time
data = InputsProducer.CreateRootDF(file_name, 0, True)
X, Y, var_pos = InputsProducer.CreateXY(data)

CPU times: user 10.2 s, sys: 170 ms, total: 10.3 s
Wall time: 10.3 s


In [19]:
def sel_acc(y_true, y_pred, n_positions, n_exp):
    pred_sorted = tf.argsort(y_pred, axis=1, direction='DESCENDING')
    #return pred_sorted[:, 0]
    n_evt = tf.shape(y_true)[0]
    evt_id = tf.range(n_evt)
    matches_vec = []
    for n in range(n_positions):
        index = tf.transpose(tf.stack([evt_id, tf.reshape(pred_sorted[:, n], shape=(n_evt,))]))
        matches_vec.append(tf.gather_nd(y_true, index))
    matches_sum = tf.add_n(matches_vec)    
    valid = tf.cast(tf.equal(matches_sum, n_exp), tf.float32)
    n_valid = tf.reduce_sum(valid)
    return n_valid / tf.cast(n_evt, tf.float32)

def sel_acc_2(y_true, y_pred):
    return sel_acc(y_true, y_pred, 2, 2)
def sel_acc_3(y_true, y_pred):
    return sel_acc(y_true, y_pred, 3, 2)
def sel_acc_4(y_true, y_pred):
    return sel_acc(y_true, y_pred, 4, 2)

In [20]:
model = HHModel(var_pos, '../config/mean_std_red.json', '../config/min_max_red.json')

2019-10-17 15:03:45.987294: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-17 15:03:46.343299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-17 15:03:46.344220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1060 3GB major: 6 minor: 1 memoryClockRate(GHz): 1.7845
pciBusID: 0000:01:00.0
2019-10-17 15:03:46.344553: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-17 15:03:46.346772: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-17 15:03:46.348679: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library lib

In [21]:
%time model.call(X[0:1,:,:])

CPU times: user 1.25 s, sys: 377 ms, total: 1.62 s
Wall time: 1.61 s


<tf.Tensor: id=478, shape=(1, 10), dtype=float32, numpy=
array([[0.76205575, 0.7586758 , 0.75255466, 0.75118256, 0.7507615 ,
        0.7470724 , 0.7442271 , 0.777784  , 0.777784  , 0.777784  ]],
      dtype=float32)>

2019-10-17 15:03:47.079428: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2019-10-17 15:03:47.918418: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


In [22]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[sel_acc_2, sel_acc_3, sel_acc_4])

In [23]:
model.build(X.shape)
model.summary()

Model: "hh_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
std_layer (StdLayer)         multiple                  0         
_________________________________________________________________
scale_layer (ScaleLayer)     multiple                  0         
_________________________________________________________________
lstm (LSTM)                  multiple                  1984      
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
concatenate (Concatenate)    multiple                  0         
_________________________________________________________________
lstm_1 (LSTM)                multiple                  3072      
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0  

In [24]:
csv_logger = CSVLogger('history.csv', append=True, separator=';')
history_callback = model.fit(X, Y[:,:,:], validation_split=0.2, epochs=10, batch_size=1000, callbacks=[csv_logger])

Train on 1888 samples, validate on 473 samples
Epoch 1/10
1888/1888 [==============================] - 10s 5ms/sample - loss: 1.2160 - sel_acc_2: 0.0175 - sel_acc_3: 0.0607 - sel_acc_4: 0.1195 - val_loss: 1.1947 - val_sel_acc_2: 0.0085 - val_sel_acc_3: 0.0486 - val_sel_acc_4: 0.1205
Epoch 2/10
1888/1888 [==============================] - 0s 29us/sample - loss: 1.2010 - sel_acc_2: 0.0169 - sel_acc_3: 0.0545 - sel_acc_4: 0.1094 - val_loss: 1.1784 - val_sel_acc_2: 0.0085 - val_sel_acc_3: 0.0486 - val_sel_acc_4: 0.1226
Epoch 3/10
1888/1888 [==============================] - 0s 30us/sample - loss: 1.1835 - sel_acc_2: 0.0156 - sel_acc_3: 0.0544 - sel_acc_4: 0.1131 - val_loss: 1.1624 - val_sel_acc_2: 0.0085 - val_sel_acc_3: 0.0486 - val_sel_acc_4: 0.1247
Epoch 4/10
1888/1888 [==============================] - 0s 31us/sample - loss: 1.1672 - sel_acc_2: 0.0184 - sel_acc_3: 0.0545 - sel_acc_4: 0.1194 - val_loss: 1.1468 - val_sel_acc_2: 0.0085 - val_sel_acc_3: 0.0486 - val_sel_acc_4: 0.1247
Epoch

2019-10-17 15:03:57.904935: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_standard_lstm_19615_20214' and '__inference___backward_cudnn_lstm_with_fallback_17895_19354_specialized_for_StatefulPartitionedCall_at___inference_distributed_function_21201' both implement 'lstm_507d93a1-51d6-4df5-85cb-6f9b2c346321' but their signatures do not match.
2019-10-17 15:04:01.370787: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_cudnn_lstm_with_fallback_23835_specialized_for_hh_model_lstm_1_StatefulPartitionedCall_at___inference_distributed_function_25587' and '__inference_cudnn_lstm_with_fallback_23835' both implement 'lstm_e6afa1f0-51ad-42d2-a064-c45b83d884fe' but their signatures do not match.


In [25]:
model.save_weights('./checkpoints/my_checkpoint')